# Enhanced RAG Pipeline for Long Documents and Complex Concept Systems

## 주요 개선사항:
1. **계층적 문서 분석**: 문서 구조를 파악하여 전체 개요와 세부사항을 구분
2. **개념 맵핑**: PNS_1, PNS_2 등의 개념들을 자동으로 식별하고 분류
3. **다단계 검색**: 전체 개요 → 세부 개념 → 구체적 내용의 단계적 검색
4. **컨텍스트 윈도우 확장**: 관련 섹션들을 그룹화하여 더 넓은 맥락 제공
5. **개요 우선 접근**: 긴 문서의 전체적인 맥락을 먼저 파악한 후 세부사항 탐색

In [93]:
# Enhanced RAG Pipeline for Long Documents and Complex Concept Systems
# - Hierarchical document analysis
# - Concept mapping and categorization  
# - Multi-stage retrieval strategy
# - Context window expansion
# - Overview-first approach for comprehensive understanding

import os, re, json
from typing import List, Dict, Any, Tuple, Optional, Set
from dataclasses import dataclass
from collections import Counter, defaultdict
import networkx as nx

from langchain.docstore.document import Document
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

try:
    from langchain_text_splitters import MarkdownHeaderTextSplitter
    HAS_MD_SPLITTER = True
except Exception:
    HAS_MD_SPLITTER = False

DATA_FILE = "data/dev_center_guide_allmd_touched.md"
EMBED_MODEL = "bge-m3:latest"
LLM_MODEL = "exaone3.5:latest"
FAISS_DIR = "models/faiss_rag_enhanced_bge-m3_hierarchical"

print("Enhanced RAG Config:")
print(f"- DATA_FILE: {DATA_FILE}")
print(f"- EMBED_MODEL: {EMBED_MODEL}")
print(f"- LLM_MODEL: {LLM_MODEL}")
print(f"- FAISS_DIR:  {FAISS_DIR}")

Enhanced RAG Config:
- DATA_FILE: data/dev_center_guide_allmd_touched.md
- EMBED_MODEL: bge-m3:latest
- LLM_MODEL: exaone3.5:latest
- FAISS_DIR:  models/faiss_rag_enhanced_bge-m3_hierarchical


In [96]:
# Enhanced RAG System 클래스들을 import
from code_dev.enhanced_rag_system import (
    HierarchicalDocumentAnalyzer, 
    MultiStageRetriever, 
    ContextWindowExpander, 
    EnhancedRAGSystem,
    COMPREHENSIVE_OVERVIEW_PROMPT,
    DETAILED_ANALYSIS_PROMPT
)

print("✅ Enhanced RAG System 모듈 로드 완료!")

Enhanced RAG Config:
- DATA_FILE: ../data/dev_center_guide_allmd_touched.md
- EMBED_MODEL: bge-m3:latest
- LLM_MODEL: exaone3.5:latest
- FAISS_DIR:  ../models/faiss_rag_enhanced_bge-m3_hierarchical
✅ Enhanced RAG System 모듈 로드 완료!


In [97]:
# Enhanced RAG System 초기화
enhanced_system = EnhancedRAGSystem(DATA_FILE, EMBED_MODEL, LLM_MODEL)

print("🚀 Enhanced RAG System이 성공적으로 초기화되었습니다!")

📊 문서 구조 분석 완료:
   - 총 노드: 494
   - 개념 그룹: 0
   - 개요 섹션: 18
🚀 Enhanced RAG System 초기화 완료!
🚀 Enhanced RAG System이 성공적으로 초기화되었습니다!


In [98]:
# 테스트 1: PNS의 전반적인 기능 요약
print("🔍 테스트 1: PNS의 전반적인 기능을 요약해주세요")
print("="*80)

query1 = "PNS의 전반적인 기능을 요약해주세요"
answer1 = enhanced_system.answer_comprehensive(query1, include_overview=True)
print(f"답변:\n{answer1}")

🔍 테스트 1: PNS의 전반적인 기능을 요약해주세요
답변:
### 분석 결과 요약

#### 📋 전체 구조
1. **웹 기반 인앱 결제 가이드**: 원스토어 플랫폼을 이용한 웹 기반 인앱 결제 시스템에 대한 포괄적인 안내서입니다.
2. **서버 API 활용**: 결제 프로세스는 서버 측 API를 통해 이루어지며, 특히 OAuth 인증과 User Access Token의 사용이 강조됩니다.
3. **HTTP 요청 및 응답**: 결제 요청과 응답은 HTTP 프로토콜을 통해 이루어지며, 특히 Header에 접근 토큰(User Access Token)의 포함이 중요합니다.
4. **표준 응답 코드**: 다양한 상황에 따른 HTTP 응답 코드와 그 의미, 적용 범위가 상세히 설명되어 있습니다.

#### 🎯 핵심 개념
1. **OAuth 인증**: 사용자 인증을 위해 OAuth 인증을 활용하며, 이로 인해 발급되는 User Access Token이 모든 결제 관련 요청에 필수적입니다.
2. **User Access Token**: 결제 요청과 응답 사이에서 사용자 권한을 확인하고 검증하는 데 사용되는 접근 토큰입니다.
3. **결제 요청 (paymentUrl + paymentParam)**: `paymentUrl`을 통해 결제 파라미터(`paymentParam`)를 전송하여 결제를 요청합니다.
4. **결제 응답 처리**: 결제 결과를 수신하고 적절히 처리하는 방법이 설명되어 있습니다.

#### 📖 기능 요약
- **결제 요청 프로세스**: 사용자 인증 후, 결제 파라미터를 포함한 요청을 서버 API에 전송합니다. 이 과정에서 User Access Token이 헤더에 포함되어야 합니다.
- **결제 응답 처리**: 결제 완료 후, 개발 서버로 결제 결과가 전달되며, 이를 적절히 처리하여 사용자에게 피드백을 제공합니다.

#### 🔍 추가 정보
- **API 호출 예시**: 구체적인 API 호출 예시나 코드 스니펫이 추가로 제공되면 더 도움이 될 것입니다.
- **오

In [99]:
# 테스트 2: PNS_1, PNS_2 등의 세부 개념들
print("🔍 테스트 2: PNS_1, PNS_2 등의 세부 개념들이 무엇인지 알려주세요")
print("="*80)

query2 = "PNS의 개념들이 무엇인지 알려주세요"
answer2 = enhanced_system.answer_comprehensive(query2, include_overview=True)
print(f"답변:\n{answer2}")

🔍 테스트 2: PNS_1, PNS_2 등의 세부 개념들이 무엇인지 알려주세요
답변:
### 📊 전체 구조

1. **개요 및 목표**
   - 문서의 목적과 주요 사용자(개발자, 사업자 등)를 명확히 설명합니다.
   
2. **서버 APIs**
   - **접근 토큰(User Access Token) 사용 방법**
     - OAuth Token을 이용한 서버 API 호출 방법을 설명합니다.
   - **결제 요청**
     - `paymentUrl`을 통해 `paymentParam`을 전달하여 결제 요청을 수행하는 과정을 설명합니다.
   - **결제 응답 처리**
     - 결제 완료 후 표준 결제화면에서의 응답을 개발사로 전달하는 방법을 설명합니다.

3. **표준 응답 코드**
   - 다양한 API 호출 결과에 대한 HTTP 상태 코드와 메시지를 상세히 나열합니다. 각 응답 코드의 의미와 발생 가능한 상황을 설명합니다.

4. **결제 프로세스 상세**
   - 결제 요청부터 응답까지의 전체 프로세스를 단계별로 설명합니다. 특히, 표준 결제화면의 역할과 결제 완료 후의 응답 처리 과정이 포함됩니다.

### 🎯 핵심 개념

1. **접근 토큰 (User Access Token)**
   - 모든 서버 API 호출 시 필요한 인증 토큰으로, OAuth 기반의 인증 시스템을 통해 발급됩니다.

2. **결제 요청 및 응답**
   - 결제 요청을 위한 `paymentUrl`과 `paymentParam`의 사용 방법 및 결제 완료 후의 응답 처리 메커니즘.

3. **HTTP 상태 코드**
   - 각 API 호출 결과에 따른 HTTP 상태 코드 (예: 200 성공, 400 잘못된 요청, 401 권한 없음 등)와 그 의미를 이해하는 것이 중요합니다.

### 📖 기능 요약

- **결제 요청**: 개발사는 발급받은 접근 토큰을 이용해 `paymentUrl`을 통해 결제 요청을 수행합니다. 이 과정에서 `paymentParam`을 전달하여 구매 정보

In [100]:
# 테스트 3: 원스토어 인앱결제의 전체적인 구조
print("🔍 테스트 3: 원스토어 인앱결제의 전체적인 구조를 설명해주세요")
print("="*80)

query3 = "원스토어 인앱결제의 전체적인 구조를 설명해주세요"
answer3 = enhanced_system.answer_comprehensive(query3, include_overview=True)
print(f"답변:\n{answer3}")

🔍 테스트 3: 원스토어 인앱결제의 전체적인 구조를 설명해주세요
답변:
### 분석 결과 요약

#### 📋 전체 구조
1. **결제 요청 (API 요청)**
   - **구현 세부사항**: REST API를 통한 결제 요청 방법 설명
   - **요청 예시**: HTTP 요청 방법, 사용 예시 및 코드 샘플 제공
   - **응답 코드**: 성공/실패 응답 코드와 그 의미 설명

2. **결제 응답 (결제 결과 처리)**
   - **구현 세부사항**: 결제 결과를 개발사로 전달하는 방법 (`returnUrl`, `callbackUrl`)
   - **요청 예시**: `returnUrl`을 통한 폼 데이터 전송 방식과 `callbackUrl`을 통한 JSON 데이터 전송 방식
   - **응답 코드**: 결제 결과에 대한 다양한 응답 코드와 그 의미 설명 (성공, 실패, 사용자 취소 등)

#### 🎯 핵심 개념
1. **REST API**: 결제 요청과 결과 전달을 위한 통신 프로토콜로서의 역할
2. **구매 토큰 (Purchase Token)**: 결제 요청 및 결과 확인 시 중요한 식별자
3. **개발자 페이로드 (Developer Payload)**: 개발자 측에서 관리하는 고유 식별자로 결제 요청 및 결과 확인에 사용
4. **서명 검증 (Signature Verification)**: 보안을 위한 데이터 무결성 및 진위 확인 메커니즘
5. **응답 코드와 메시지**: 결제 프로세스의 다양한 상태를 나타내는 코드와 메시지의 중요성

#### 📖 기능 요약
- **결제 요청**: 개발자는 구매 정보를 포함한 결제 요청을 REST API를 통해 수행한다. 이 요청은 구매 토큰, 개발자 페이로드 등의 요소를 포함한다.
- **결제 응답 처리**: 결제 결과는 두 가지 방식으로 전달된다: 
   - **returnUrl**: 사용자를 리다이렉트하여 결제 결과를 보여주는 URL을 통해 정보를 전달.
   - **callbackUrl**: JSON 형태로 결제

In [101]:
# 테스트 4: 세부적인 필드값 질문 (기존 방식과 비교)
print("🔍 테스트 4: PNS의 purchaseState 값은 무엇인가요? (상세 분석)")
print("="*80)

query4 = "PNS의 purchaseState 값은 무엇인가요?"
answer4 = enhanced_system.answer_comprehensive(query4, include_overview=False)
print(f"답변:\n{answer4}")

🔍 테스트 4: PNS의 purchaseState 값은 무엇인가요? (상세 분석)
답변:
### 🛠 요약 및 주요 내용

#### 🛠 결제 응답 분석

**📊 정확한 값들:**
- **응답 코드 (HTTP Status Code):** 주로 `200 OK` (성공) 또는 오류 상황에 따른 `4xx` 및 `5xx` 범위의 코드들이 사용됩니다. 예를 들어:
  - `200 OK`: 성공적인 결제 요청 처리.
  - `400 Bad Request`: 잘못된 요청 파라미터로 인한 실패.
  - `401 Unauthorized`: 접근 토큰이 유효하지 않거나 만료된 경우.
  - `404 Not Found`: 요청한 리소스가 존재하지 않는 경우.
  - `409 Conflict`: 구매 상태나 권한 관련 오류.
  - `500 Internal Server Error`: 서버 측 오류.

**💡 구체적 예시:**
- **성공 사례:**
  ```json
  {
    "responseCode": "OK",
    "paymentResult": {
      "transactionId": "TX123456789",
      "productId": "PROD987654",
      "status": "SUCCESS",
      "purchaseAmount": "10000",
      "currency": "KRW",
      "purchaseDate": "2023-10-05T12:00:00Z"
    }
  }
  ```
  - 위 예시에서 `responseCode: "OK"`는 결제 요청이 성공적으로 처리되었음을 나타냅니다. `status: "SUCCESS"` 또한 결제 상태가 성공임을 명시합니다.

- **오류 사례:**
  ```json
  {
    "responseCode": "BAD_REQUEST",
    "errorMessage": "Invalid User Access Token",
    "details": {
      "errorCode": "In

In [ ]:
# 문서 구조 분석 결과 확인
print("📊 문서 구조 분석 결과")
print("="*80)

analyzer = enhanced_system.analyzer
nodes = enhanced_system.multi_stage_retriever.nodes
concept_hierarchy = enhanced_system.multi_stage_retriever.concept_hierarchy
overview_sections = enhanced_system.multi_stage_retriever.overview_sections

print(f"총 노드 수: {len(nodes)}")
print(f"개념 그룹 수: {len(concept_hierarchy)}")
print(f"개요 섹션 수: {len(overview_sections)}")

print("\n🎯 발견된 개념 그룹들:")
for base_concept, concept_nodes in concept_hierarchy.items():
    print(f"  - {base_concept}: {len(concept_nodes)}개 노드")
    for node_id in concept_nodes[:3]:  # 처음 3개만 표시
        node = nodes[node_id]
        print(f"    * {node.title}")

print("\n📖 개요 섹션들:")
for node_id in overview_sections[:5]:  # 처음 5개만 표시
    node = nodes[node_id]
    print(f"  - {node.title}")

In [ ]:
# 사용자 정의 쿼리 테스트
def test_custom_query(query: str, include_overview: bool = True):
    """사용자 정의 쿼리를 테스트하는 함수"""
    print(f"🔍 쿼리: {query}")
    print("-"*60)
    
    try:
        answer = enhanced_system.answer_comprehensive(query, include_overview=include_overview)
        print(f"답변:\n{answer}")
    except Exception as e:
        print(f"오류 발생: {e}")
    
    print("="*80)

# 사용 예시
# test_custom_query("원스토어 인앱결제에서 결제 상태는 어떻게 확인하나요?")
# test_custom_query("PNS 메시지의 구조를 자세히 설명해주세요", include_overview=False)